In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout 
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
import tensorflow as tf
import cv2
import os
import numpy as np
import random
from sklearn import metrics

In [ ]:
labels = ['pyramid', 'Tetrahedron','cow','cube']
img_size = 600
def get_data(data_dir,data_size):
    data = [] 
    for label in labels: 
        path = os.path.join(data_dir, label)
        class_num = labels.index(label)
        for i in range(data_size):
            img=os.listdir(path)[i]
            try:
                img_arr = cv2.imread(os.path.join(path, img),cv2.IMREAD_GRAYSCALE)
                resized_arr = cv2.resize(img_arr, (img_size, img_size)) #Reshaping images to preferred size
                data.append([resized_arr, class_num])
            except Exception as e:
                print(e)
    return np.array(data)

In [ ]:
data_set = get_data(r'.\Data',20)

In [ ]:
X=[]
y=[]
for feature, label in data_set:
    X.append(feature)
    y.append(label)

In [ ]:
#Spliting the Data 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

In [ ]:
#test 
num = random.randint(0,len(X_train))
plt.figure(figsize = (5,5))
plt.imshow(X_train[num],cmap='gray')
plt.title(y_train[num])
print(num)

In [ ]:
# Normalize the data

X_train = np.array(X_train) / 255
X_test = np.array(X_test) / 255

#reshape and convert target values to one hot vectors
X_train=X_train.reshape(-1, img_size, img_size, 1)
y_train = np.array(y_train)
y_train = keras.utils.to_categorical(y_train, len(labels))


X_test=X_test.reshape(-1, img_size, img_size, 1)
y_test = np.array(y_test)
y_test = keras.utils.to_categorical(y_test, len(labels))

print(np.shape(X_train))
print(np.shape(y_train))
print(np.shape(X_test))
print(np.shape(y_test))

In [ ]:
#test
num = random.randint(0,len(X_train))
plt.figure(figsize = (5,5))
plt.imshow(X_train[num],cmap='gray')
plt.title(y_train[num])
print(num)

## CNN Model

In [ ]:
model = Sequential()
model.add(Conv2D(32,3,padding="same", activation="relu", input_shape=(600,600,1)))
model.add(MaxPool2D())

model.add(Conv2D(32, 3, padding="same", activation="relu"))
model.add(MaxPool2D())

model.add(Conv2D(64, 3, padding="same", activation="relu"))
model.add(MaxPool2D())
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(128,activation="relu"))
model.add(Dense(len(labels), activation="softmax"))

model.summary()

In [ ]:
opt = Adam(learning_rate=0.000001)
model.compile(optimizer = opt , loss=keras.losses.categorical_crossentropy , metrics = ['accuracy'])

In [ ]:
epochs = 20
history = model.fit(X_train,y_train,epochs = epochs , validation_data = (X_test, y_test))

## Accuracy and Loss Plots

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(15, 15))
plt.subplot(2, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

## Classification report

In [ ]:
y_pred = model.predict(X_test)
# convert one hot to vector
Y_test = np.argmax(y_test, axis = 1)
Y_pred = np.argmax(y_pred, axis = 1)

print(classification_report(Y_test, Y_pred, target_names = labels))


## Test loss: and Test accuracy

In [ ]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss: {:.4}'.format(score[0]))
print('Test accuracy: {:.4}'.format(score[1]))

## Confusion matrix

In [ ]:
# confusion matrix


y_pred = model.predict(X_test)
# convert one hot to vector
Y_test = np.argmax(y_test, axis = 1)
Y_pred = np.argmax(y_pred, axis = 1)

cm1 = metrics.confusion_matrix(Y_test, Y_pred)

plt.figure(figsize=(9,9))
ax= plt.subplot()
sns.heatmap(cm1, annot=True, fmt=".0f", linewidths=.5, square=True, cmap='Blues_r')
ax.set_ylabel('true shape', size=17)
ax.set_xlabel('predicted shape', size=17)
ax.xaxis.set_ticklabels(['pyramid', 'Tetrahedron','cow','cube']); ax.yaxis.set_ticklabels(['pyramid', 'Tetrahedron','cow','cube'])


## Showing all wrong predictions


In [ ]:
errors = (Y_pred - Y_test != 0)
error_pos = np.array(np.where(errors))
print('total number of errors:', len(error_pos[0,:]))

errl=[]
for i in range(len(Y_test)):
    if Y_pred[i] - Y_test[i] != 0:
        errl.append(i)
        
plt.figure(figsize=(15,15))
x, y =3,int(len(errl)/3)+1
for i in range(len(errl)):
    plt.subplot(y, x,i+1)
    m=int(errl[i])
    plt.imshow(X_test[m],cmap='gray')
    plt.title('target: {}\npredicted: {}'.format((labels[Y_test[m]]),labels[Y_pred[m]]))
    plt.tight_layout()
    plt.axis('off')
plt.show()

## Showing some good predictions

In [ ]:

plt.figure(figsize=(12,12))
x, y =5,5
for i in range(25):
    plt.subplot(y, x,i+1)
    m=random.randint(0, 50)
    while m in errl:
        m=random.randint(0, 200)
    plt.imshow(X_test[m],cmap='gray')
    plt.title('target: {}\npredicted: {}'.format((labels[Y_test[m]]),labels[Y_pred[m]]))
    plt.tight_layout()
    plt.axis('off')

plt.show()